In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357278 sha256=8c598854007e75c2e9484f34527fbef2cdd43fa487e499f2b9207ce84e8f2c91
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
!pip install xgboost

In [ ]:
import pandas as pd

df = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')




In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.4, random_state=42)

algo = SVD(n_factors=10, biased=False)
algo.fit(trainset)

predictions = algo.test(testset)

mse = accuracy.mse(predictions)
mae=accuracy.mae(predictions)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

MSE: 0.7912
MAE:  0.7017
Mean Squared Error: 0.7911611020677853
Mean Absolute Error: 0.7017171681177907


In [ ]:
#svd + xgboost
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split as train_test_split_s

df_ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')
df_movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movieId', 'title', 'genres'], engine='python', encoding='ISO-8859-1')

df = pd.merge(df_ratings, df_movies, on='movieId')

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.4, random_state=42)

trainset_g, testset_g = train_test_split_s(df, test_size=0.4, random_state=42)

genres_list = list(set("|".join(trainset_g['genres'].unique()).split('|')))

user_genre_ratings = {genre: {} for genre in genres_list}

for userId in trainset_g['userId'].unique():
    user_data = trainset_g[trainset_g['userId'] == userId]
    for genre in genres_list:
        genre_ratings = user_data[user_data['genres'].str.contains(genre)]['rating']
        if not genre_ratings.empty:
            user_genre_ratings[genre][userId] = genre_ratings.mean()
        else:
            user_genre_ratings[genre][userId] = np.nan

user_genre_df = pd.DataFrame(user_genre_ratings)



In [ ]:
user_genre_df

,Crime,Fantasy,Documentary,Sci-Fi,Western,Adventure,Thriller,Horror,Children's,Animation,Musical,Film-Noir,Comedy,Romance,War,Drama,Action,Mystery
4591,3.238095,3.565217,3.0,3.328358,3.000000,3.350000,3.207547,2.641026,3.285714,3.620690,3.400000,NaN,3.522727,3.326531,3.240000,3.411765,3.284404,3.000000
5954,3.864865,3.480000,NaN,3.436170,3.636364,3.031915,3.434783,3.135922,3.187500,3.333333,3.303030,4.230769,3.141243,2.880597,3.583333,3.520468,2.893617,3.695652
3816,4.181818,3.533333,5.0,3.640000,4.571429,3.928571,4.272727,3.714286,3.914286,3.941176,4.176471,4.000000,3.945652,3.966667,4.000000,3.978723,4.072727,4.000000
5788,2.928571,3.000000,NaN,2.812500,2.714286,2.764706,2.652778,2.505495,2.552632,4.555556,3.666667,NaN,2.910781,2.760870,3.333333,3.564103,2.537572,2.454545
5951,3.833333,5.000000,NaN,4.111111,NaN,4.333333,3.833333,4.500000,4.000000,4.333333,4.000000,4.000000,4.571429,4.750000,4.400000,3.555556,4.200000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,4.000000,3.500000,4.000000,NaN
723,5.000000,NaN,NaN,4.000000,NaN,4.000000,2.500000,NaN,NaN,NaN,NaN,NaN,4.000000,3.000000,4.000000,4.000000,3.666667,NaN
5804,NaN,NaN,NaN,4.000000,3.000000,3.000000,4.000000,NaN,NaN,NaN,NaN,NaN,3.750000,4.200000,4.000000,4.000000,NaN,4.000000
1174,1.000000,NaN,NaN,3.000000,NaN,3.000000,2.500000,1.000000,NaN,NaN,NaN,NaN,3.333333,3.000000,NaN,3.400000,2.000000,4.000000


In [ ]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(trainset_g[['userId', 'movieId', 'rating']], reader)

trainset_1, trainset_2 = train_test_split(data, test_size=0.5, random_state=42)

algo = SVD(n_factors=4, n_epochs=80, biased=False)
algo.fit(trainset_1)

predictions = algo.test(trainset_2)

svd_predictions = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions], columns=['userId', 'movieId', 'svd_pred'])

test_df = pd.DataFrame(trainset_2, columns=['userId', 'movieId', 'rating'])
test_df = pd.merge(test_df, df_movies, on='movieId')
test_df = pd.merge(test_df, svd_predictions, on=['userId', 'movieId'])

def get_avg_genre_rating(row):
    user_genres = row['genres'].split('|')
    avg_genre_rating = np.nanmean([user_genre_df.at[row['userId'], genre] for genre in user_genres])
    return avg_genre_rating

test_df['avg_genre_rating'] = test_df.apply(get_avg_genre_rating, axis=1)

test_df.dropna(subset=['avg_genre_rating'], inplace=True)

X_train = test_df[['svd_pred', 'avg_genre_rating']]
y_train = test_df['rating']
xgb = XGBRegressor(objective='reg:squarederror',reg_alpha=0.4,max_depth=10)
xgb.fit(X_train, y_train)

predictions_f = algo.test(testset)

final_svd_predictions = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions_f], columns=['userId', 'movieId', 'svd_pred'])

test_df_f = pd.DataFrame(testset, columns=['userId', 'movieId', 'rating'])
test_df_f = pd.merge(test_df_f, df_movies, on='movieId')
test_df_f = pd.merge(test_df_f, final_svd_predictions, on=['userId', 'movieId'])

test_df_f['avg_genre_rating'] = test_df_f.apply(get_avg_genre_rating, axis=1)

test_df_f.dropna(subset=['avg_genre_rating'], inplace=True)

X_test = test_df_f[['svd_pred', 'avg_genre_rating']]
y_test = test_df_f['rating']

final=xgb.predict(X_test)

mse = mean_squared_error(final, y_test)
mae = mean_absolute_error(final, y_test)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

Mean Squared Error: 0.693422333548527
Mean Absolute Error: 0.6562666739725659


In [ ]:
#with mean and variance
#svd + xgboost

df_ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')
df_movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movieId', 'title', 'genres'], engine='python', encoding='ISO-8859-1')

df = pd.merge(df_ratings, df_movies, on='movieId')

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.4, random_state=42)
trainset_g, testset_g = train_test_split_s(df, test_size=0.4, random_state=42)

genres_list = list(set("|".join(trainset_g['genres'].unique()).split('|')))
user_genre_ratings = {genre: {} for genre in genres_list}

for userId in trainset_g['userId'].unique():
    user_data = trainset_g[trainset_g['userId'] == userId]
    for genre in genres_list:
        genre_ratings = user_data[user_data['genres'].str.contains(genre)]['rating']
        if not genre_ratings.empty:
            user_genre_ratings[genre][userId] = genre_ratings.mean()
        else:
            user_genre_ratings[genre][userId] = np.nan

user_genre_df = pd.DataFrame(user_genre_ratings)


In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from xgboost import XGBRegressor
import numpy as np
import pandas as pd

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(trainset_g[['userId', 'movieId', 'rating']], reader)

trainset_1, trainset_2 = train_test_split(data, test_size=0.5, random_state=42)

algo = SVD(n_factors=4, n_epochs=80, biased=False)
algo.fit(trainset_1)

predictions = algo.test(trainset_2)

svd_predictions = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions], columns=['userId', 'movieId', 'svd_pred'])

test_df = pd.DataFrame(trainset_2, columns=['userId', 'movieId', 'rating'])
test_df = pd.merge(test_df, df_movies, on='movieId')
test_df = pd.merge(test_df, svd_predictions, on=['userId', 'movieId'])

def get_avg_genre_rating(row):
    user_genres = row['genres'].split('|')
    avg_genre_rating = np.nanmean([user_genre_df.at[row['userId'], genre] for genre in user_genres])
    return avg_genre_rating

test_df['avg_genre_rating'] = test_df.apply(get_avg_genre_rating, axis=1)
test_df.dropna(subset=['avg_genre_rating'], inplace=True)

movie_stats = trainset_g.groupby('movieId')['rating'].agg(['mean', 'var']).reset_index()
movie_stats.columns = ['movieId', 'mean_rating', 'rating_variance']
test_df = pd.merge(test_df, movie_stats, on='movieId')

X_train = test_df[['svd_pred', 'avg_genre_rating', 'mean_rating', 'rating_variance']]
y_train = test_df['rating']
xgb = XGBRegressor(objective='reg:squarederror', reg_alpha=0.4, max_depth=6)
xgb.fit(X_train, y_train)

predictions_f = algo.test(testset)

final_svd_predictions = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions_f], columns=['userId', 'movieId', 'svd_pred'])

test_df_f = pd.DataFrame(testset, columns=['userId', 'movieId', 'rating'])
test_df_f = pd.merge(test_df_f, df_movies, on='movieId')
test_df_f = pd.merge(test_df_f, final_svd_predictions, on=['userId', 'movieId'])

test_df_f['avg_genre_rating'] = test_df_f.apply(get_avg_genre_rating, axis=1)
test_df_f.dropna(subset=['avg_genre_rating'], inplace=True)

test_movie_stats = test_df_f.groupby('movieId')['rating'].agg(['mean', 'var']).reset_index()
test_movie_stats.columns = ['movieId', 'mean_rating', 'rating_variance']
test_df_f = pd.merge(test_df_f, test_movie_stats, on='movieId')




In [ ]:
# Include the additional features in the training set
X_train = test_df[['svd_pred', 'avg_genre_rating', 'mean_rating', 'rating_variance']]
y_train = test_df['rating']
xgb = XGBRegressor(objective='reg:squarederror', reg_lambda=10, max_depth=200, gamma=0.1)
xgb.fit(X_train, y_train)

predictions_f = algo.test(testset)

final_svd_predictions = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions_f], columns=['userId', 'movieId', 'svd_pred'])

test_df_f = pd.DataFrame(testset, columns=['userId', 'movieId', 'rating'])
test_df_f = pd.merge(test_df_f, df_movies, on='movieId')
test_df_f = pd.merge(test_df_f, final_svd_predictions, on=['userId', 'movieId'])

test_df_f['avg_genre_rating'] = test_df_f.apply(get_avg_genre_rating, axis=1)
test_df_f.dropna(subset=['avg_genre_rating'], inplace=True)

test_movie_stats = test_df_f.groupby('movieId')['rating'].agg(['mean', 'var']).reset_index()
test_movie_stats.columns = ['movieId', 'mean_rating', 'rating_variance']
test_df_f = pd.merge(test_df_f, test_movie_stats, on='movieId')

X_test = test_df_f[['svd_pred', 'avg_genre_rating', 'mean_rating', 'rating_variance']]
y_test = test_df_f['rating']

final_xgb = xgb.predict(X_test)

mse = mean_squared_error(final_xgb, y_test)
mae = mean_absolute_error(final_xgb, y_test)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


Mean Squared Error: 0.6760099960618051
Mean Absolute Error: 0.638418758410767


In [ ]:
#knn Include the additional features in the training set
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

X_train = test_df[['svd_pred', 'avg_genre_rating', 'mean_rating', 'rating_variance']]
y_train = test_df['rating']

knnpipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('knn', KNeighborsRegressor(n_neighbors=20, weights='distance'))
])
knnpipe.fit(X_train, y_train)

predictions_f = algo.test(testset)

final_svd_predictions = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions_f], columns=['userId', 'movieId', 'svd_pred'])

test_df_f = pd.DataFrame(testset, columns=['userId', 'movieId', 'rating'])
test_df_f = pd.merge(test_df_f, df_movies, on='movieId')
test_df_f = pd.merge(test_df_f, final_svd_predictions, on=['userId', 'movieId'])

test_df_f['avg_genre_rating'] = test_df_f.apply(get_avg_genre_rating, axis=1)
test_df_f.dropna(subset=['avg_genre_rating'], inplace=True)

test_movie_stats = test_df_f.groupby('movieId')['rating'].agg(['mean', 'var']).reset_index()
test_movie_stats.columns = ['movieId', 'mean_rating', 'rating_variance']

test_df_f = pd.merge(test_df_f, test_movie_stats, on='movieId')

X_test = test_df_f[['svd_pred', 'avg_genre_rating', 'mean_rating', 'rating_variance']]
y_test = test_df_f['rating']

final_knn = knnpipe.predict(X_test)

mse = mean_squared_error(final_knn, y_test)
mae = mean_absolute_error(final_knn, y_test)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


Mean Squared Error: 0.6793087716285415
Mean Absolute Error: 0.6467109990524935


In [ ]:
#regression Include the additional features in the training set
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error

X_train = test_df[['svd_pred', 'avg_genre_rating', 'mean_rating', 'rating_variance']]
y_train = test_df['rating']

lr = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('lasso', Lasso(alpha=0.01))
])
lr.fit(X_train, y_train)

predictions_f = algo.test(testset)

final_svd_predictions = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions_f], columns=['userId', 'movieId', 'svd_pred'])

test_df_f = pd.DataFrame(testset, columns=['userId', 'movieId', 'rating'])
test_df_f = pd.merge(test_df_f, df_movies, on='movieId')
test_df_f = pd.merge(test_df_f, final_svd_predictions, on=['userId', 'movieId'])

test_df_f['avg_genre_rating'] = test_df_f.apply(get_avg_genre_rating, axis=1)
test_df_f.dropna(subset=['avg_genre_rating'], inplace=True)

test_movie_stats = test_df_f.groupby('movieId')['rating'].agg(['mean', 'var']).reset_index()
test_movie_stats.columns = ['movieId', 'mean_rating', 'rating_variance']

test_df_f = pd.merge(test_df_f, test_movie_stats, on='movieId')

X_test = test_df_f[['svd_pred', 'avg_genre_rating', 'mean_rating', 'rating_variance']]
y_test = test_df_f['rating']

final_lr = lr.predict(X_test)

mse = mean_squared_error(final_lr, y_test)
mae = mean_absolute_error(final_lr, y_test)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

Mean Squared Error: 0.706423323763787
Mean Absolute Error: 0.663512970761762


In [ ]:
# averaging two previous models
avg_pred = (final_knn + final_xgb) / 2
mse_avg = mean_squared_error(avg_pred, y_test)
mae_avg = mean_absolute_error(avg_pred, y_test)

print("Mean Squared Error (Average Prediction):", mse_avg)
print("Mean Absolute Error (Average Prediction):", mae_avg)

Mean Squared Error (Average Prediction): 0.6546344689497993
Mean Absolute Error (Average Prediction): 0.6327600814296374


In [ ]:
df = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')

In [ ]:
df

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
pivot_df = df.pivot(index='userId', columns='movieId', values='rating').fillna(0)

sparse_matrix = csr_matrix(pivot_df.values)

cosine_sim = cosine_similarity(sparse_matrix)

cosine_sim_df = pd.DataFrame(cosine_sim, index=pivot_df.index, columns=pivot_df.index)

print(cosine_sim_df)

userId      1         2         3         4         5         6         7     \
userId                                                                         
1       1.000000  0.096382  0.120610  0.132455  0.090158  0.179222  0.059678   
2       0.096382  1.000000  0.151479  0.171176  0.114394  0.100865  0.305787   
3       0.120610  0.151479  1.000000  0.151227  0.062907  0.074603  0.138332   
4       0.132455  0.171176  0.151227  1.000000  0.045094  0.013529  0.130339   
5       0.090158  0.114394  0.062907  0.045094  1.000000  0.047449  0.126257   
...          ...       ...       ...       ...       ...       ...       ...   
6036    0.186329  0.228241  0.143264  0.170583  0.293365  0.093583  0.122441   
6037    0.135979  0.206274  0.107744  0.127464  0.172686  0.065788  0.111673   
6038    0.000000  0.066118  0.120234  0.062907  0.020459  0.065711  0.000000   
6039    0.174604  0.066457  0.094675  0.064634  0.027689  0.167303  0.014977   
6040    0.133590  0.218276  0.133144  0.

In [ ]:
def top_similar_users(user_id, cosine_sim_df):
    similar_users = cosine_sim_df[user_id].sort_values(ascending=False)
    similar_users = similar_users[similar_users.index != user_id]
    top_10_similar_users = similar_users.head(10)
    return top_10_similar_users

user_id = 1010
top_similar = top_similar_users(user_id, cosine_sim_df)
print("Top 10 similar users for user", user_id, ":", top_similar)


Top 10 similar users for user 1010 : userId
3391    0.633395
5367    0.629279
4312    0.623373
1264    0.621605
4543    0.618963
3471    0.613797
3808    0.608444
710     0.607893
1579    0.601443
3507    0.594767
Name: 1010, dtype: float64


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_movie_similarity(pivot_df):
    item_user_matrix = pivot_df.T
    movie_similarity = cosine_similarity(item_user_matrix)
    return movie_similarity

movie_similarity = calculate_movie_similarity(pivot_df)

def create_movie_similarity_dict(movie_similarity, pivot_df):
    movie_sim_dict = {}
    for i, movie_id in enumerate(pivot_df.columns):
        similar_movies = [(pivot_df.columns[j], movie_similarity[i, j]) for j in range(len(pivot_df.columns)) if j != i]
        similar_movies.sort(key=lambda x: x[1], reverse=True)
        top_similar_movies = similar_movies[:10]
        movie_sim_dict[movie_id] = top_similar_movies
    return movie_sim_dict

movie_similarity_dict = create_movie_similarity_dict(movie_similarity, pivot_df)

movie_id = 914
top_similar = movie_similarity_dict[movie_id]
print("Top 10 similar movies for movie", movie_id, ":", top_similar)



Top 10 similar movies for movie 914 : [(1947, 0.5976220308356889), (2565, 0.526525312772155), (899, 0.5229031008251807), (1035, 0.5107470688517879), (1028, 0.5002571912198054), (2080, 0.4578733361409532), (594, 0.4555162611850119), (900, 0.45551564120627536), (920, 0.4490128966373095), (919, 0.4485731493962733)]
